In [1]:
import sys
sys.path.extend(['/Users/dlindenbaum/cosmiQGit/spaceNetUtilities_DaveL/python'])
#from spaceNetUtilities import evalToolsGPD as eT
#from spaceNetUtilities import geoToolsGPD as gT

In [2]:
import geopandas as gpd
#srcTruthVector = '/Users/dlindenbaum/dataStorage/spaceNetPrivate_Truth/AOI_2_Vegas/srcData/vectorData/Vegas_Footprints.shp'
srcTestVector1  = '/Users/dlindenbaum/dataStorage/xd_Vegas/15OCT22183656-S2AS_R1C2-056155973080_01_P001.shp'
srcTestVector2 = '/Users/dlindenbaum/dataStorage/xd_Vegas/15OCT22183656-S2AS_R1C3-056155973080_01_P001.shp'
srcTileIndex = '/Users/dlindenbaum/dataStorage/xd_Vegas/mul-pansharpen.shp'

In [3]:
#truthDF = gpd.read_file(srcTruthVector)
testDF1 = gpd.read_file(srcTestVector1)
testDF2 = gpd.read_file(srcTestVector2)
srcTileDF = gpd.read_file(srcTileIndex)

In [4]:
testIndex1 = testDF1.sindex
testIndex2 = testDF2.sindex

In [38]:
srcTindex1 = srcTileDF.loc[srcTileDF.contains(testDF1.geometry.values[0].centroid)]
srcTindex2 = srcTileDF.loc[srcTileDF.contains(testDF2.geometry.values[0].centroid)]

In [6]:
testDF1['location'] = srcTindex1['location'].values[0]
testDF2['location'] = srcTindex2['location'].values[0]
testDF1['index1'] = testDF1.index
testDF2['index2'] = testDF2.index

In [7]:
from shapely.geometry import LineString
coordsList = list(srcTindex2.geometry.values[0].exterior.coords)
lineStringList = []
for idx in range(1, len(coordsList)):
    lineStringList.append(LineString([coordsList[idx-1], coordsList[idx]]))

pixelDimension = 0.0000027
numPixel = 5

for lineboundary in lineStringList:
    print("Edge")
    print('test1 Length of Matchs = {}'.format(len(list(testIndex1.intersection(lineboundary.buffer(numPixel*pixelDimension).bounds)))))
    print('test2 Length of Matchs = {}'.format(len(list(testIndex2.intersection(lineboundary.buffer(numPixel*pixelDimension).bounds)))))

Edge
test1 Length of Matchs = 0
test2 Length of Matchs = 0
Edge
test1 Length of Matchs = 0
test2 Length of Matchs = 36
Edge
test1 Length of Matchs = 0
test2 Length of Matchs = 45
Edge
test1 Length of Matchs = 41
test2 Length of Matchs = 29


In [8]:
lineStringList[3].wkt

'LINESTRING (-115.2632808 36.24184080000634, -115.2632808 36.2639592)'

In [25]:
lineboundary = lineStringList[3]
numPixel = 2
testDF1Intersection = testDF1.iloc[list(testIndex1.intersection(lineboundary.buffer(numPixel*pixelDimension).bounds))]
testDF2Intersection = testDF2.iloc[list(testIndex2.intersection(lineboundary.buffer(numPixel*pixelDimension).bounds))]
testDF1Intersection.geometry = testDF1Intersection.buffer(numPixel*pixelDimension)
testDF2Intersection.geometry = testDF2Intersection.buffer(numPixel*pixelDimension)

In [26]:
res_union = gpd.overlay(testDF1Intersection, testDF2Intersection, how='union')
res_intersection = gpd.overlay(testDF1Intersection, testDF2Intersection, how='intersection')

In [27]:
print("union shape {}".format(res_union[(res_union['raster_val']==1.0) & (res_union['raster_val_2']==1.0)].shape))
res_union[(res_union['raster_val']==1.0) & (res_union['raster_val_2']==1.0)].head().area

union shape (23, 5)


1     2.145807e-09
5     8.390497e-10
15    8.123570e-10
21    2.279853e-10
24    9.676290e-11
dtype: float64

In [28]:
print("intersection shape {}".format(res_intersection[(res_intersection['raster_val']==1.0) & (res_intersection['raster_val_2']==1.0)].shape))
res_intersection[(res_intersection['raster_val']==1.0) & (res_intersection['raster_val_2']==1.0)].head().area

intersection shape (23, 5)


0    2.145807e-09
1    8.390497e-10
2    8.123570e-10
3    2.279853e-10
4    9.676290e-11
dtype: float64

In [29]:
res_union.head()

,geometry,location,location_2,raster_val,raster_val_2
0,"POLYGON ((-115.2633455998505 36.2419426698028,...",15OCT22183656-S2AS_R1C2-056155973080_01_P001.TIF,None,1.0,NaN
1,POLYGON ((-115.2632862000001 36.24283980000606...,15OCT22183656-S2AS_R1C2-056155973080_01_P001.TIF,15OCT22183656-S2AS_R1C3-056155973080_01_P001.TIF,1.0,1.0
2,POLYGON ((-115.2632821499251 36.24284502199291...,15OCT22183656-S2AS_R1C2-056155973080_01_P001.TIF,None,1.0,NaN
3,POLYGON ((-115.2632821499251 36.24284502199291...,None,15OCT22183656-S2AS_R1C3-056155973080_01_P001.TIF,NaN,1.0
4,"POLYGON ((-115.2632800697965 36.243042300006, ...",None,15OCT22183656-S2AS_R1C3-056155973080_01_P001.TIF,NaN,1.0


In [33]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [36]:
res_union[(res_union['raster_val']==1.0) & (res_union['raster_val_2']==1.0)].head().plot()

In [37]:
res_union.plot()